# 03-思考模型

参考：https://platform.moonshot.cn/docs/guide/use-kimi-k2-thinking-model

**重要**：必须使用 `getattr()` 获取 `reasoning_content`

In [ ]:
from openai import OpenAI
import json
import os

client = OpenAI(
    api_key=os.getenv("MOONSHOT_API_KEY", "your-api-key"),
    base_url="https://api.moonshot.cn/v1",
)

## kimi-k2-thinking（强制思考）

In [ ]:
# 来自官方文档的示例
response = client.chat.completions.create(
    model="kimi-k2-thinking",
    messages=[
        {"role": "system", "content": "你是 Kimi。"},
        {"role": "user", "content": "请解释 1+1=2。"}
    ]
)

message = response.choices[0].message

# 使用 getattr 获取 reasoning_content
reasoning = getattr(message, "reasoning_content", None)
content = message.content

if reasoning:
    print("思考过程：")
    print("=" * 60)
    print(reasoning)
    print("=" * 60)
    print()

print("最终答案：")
print(content)

## kimi-k2.5（默认思考）

In [ ]:
# kimi-k2.5 默认启用思考能力
response = client.chat.completions.create(
    model="kimi-k2.5",
    messages=[
        {"role": "user", "content": "请解释 1+1=2。"}
    ]
)

message = response.choices[0].message
reasoning = getattr(message, "reasoning_content", None)

if reasoning:
    print(f"思考过程 ({len(reasoning)} 字符)：")
    print(reasoning[:500] + "..." if len(reasoning) > 500 else reasoning)
    print()

print(f"最终答案: {message.content}")

## 流式输出（思考模型）

In [ ]:
response = client.chat.completions.create(
    model="kimi-k2-thinking",
    messages=[{"role": "user", "content": "解释量子计算"}],
    stream=True
)

print("思考过程：")
print("-" * 60)

in_content = False

for chunk in response:
    delta = chunk.choices[0].delta
    
    # 获取 reasoning_content
    reasoning = getattr(delta, "reasoning_content", None)
    if reasoning:
        print(reasoning, end="", flush=True)
    
    # content 在 reasoning_content 之后出现
    if delta.content:
        if not in_content:
            in_content = True
            print("\n\n最终答案：")
            print("-" * 60)
        print(delta.content, end="", flush=True)

print()

## 多步工具调用（思考模型）

**重要配置**：
- `max_tokens >= 16000`
- `temperature=1.0`
- `stream=True` 推荐

In [ ]:
# 来自官方文档的多步工具调用示例
tools = [
    {
        "type": "function",
        "function": {"name": "date", "description": "获取当前日期"}
    },
    {
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "搜索网页内容",
            "parameters": {
                "type": "object",
                "properties": {"query": {"type": "string"}},
                "required": ["query"]
            }
        }
    }
]

response = client.chat.completions.create(
    model="kimi-k2-thinking",
    messages=[
        {"role": "system", "content": "你是一个助手，负责生成今日新闻报告。"},
        {"role": "user", "content": "请生成今日的新闻报告。"}
    ],
    tools=tools,
    max_tokens=16000,
    temperature=1.0,
    stream=True
)

print("多步工具调用演示：\n")
for chunk in response:
    delta = chunk.choices[0].delta
    
    reasoning = getattr(delta, "reasoning_content", None)
    if reasoning:
        print(f"[思考] {reasoning}", end="")
    
    if delta.content:
        print(delta.content, end="")
    
    if hasattr(delta, "tool_calls") and delta.tool_calls:
        print(f"\n[工具调用] {delta.tool_calls}")

print()